In [0]:
# JDBC URL
jdbc_url = "jdbc:sqlserver://doanquanglam.database.windows.net:1433;database=BigdataCuoiKy-DoanQuangLam"

# Nếu dùng trực tiếp username/password (không khuyến khích trong production)
username = "CloudSAe8cbab11"  # thay bằng tên người dùng của bạn
password = "@Doanquanglam281"  # thay bằng mật khẩu thực tế

In [0]:
spark.conf.set(
  "fs.azure.account.key.doanquanglam.blob.core.windows.net",
  "6b+5X1whqm5IG8eQDg/0AAIU5tiu3aEXgmqJrBheRTjmmnS80mCFb7yeirJXZf16oYfV97wdwZJx+AStb2G5RQ=="
)



In [0]:
dbutils.widgets.text("blob_path", "")
blob_path = dbutils.widgets.get("blob_path")


In [0]:
#  xử lý chuỗi để spark có thể đọc được file 
# Chuyển đổi sang định dạng wasbs
from urllib.parse import urlparse
# http://doanquanglam.blob.core.windows.net/rawdata/234/test_data.json
# thành
# wasbs://rawdata@doanquanglam.blob.core.windows.net/234/test_data.json
parsed_url = urlparse(blob_path)
container = parsed_url.path.split('/')[1]
path_in_container = '/'.join(parsed_url.path.split('/')[2:])
account = parsed_url.netloc.split('.')[0]
wasbs_path = f"wasbs://{container}@{account}.blob.core.windows.net/{path_in_container}"

df = spark.read.format("parquet").load(wasbs_path)

In [0]:
# Ghi vào bảng tên là 'earthquake_results'
df.write \
  .format("jdbc") \
  .option("url", jdbc_url) \
  .option("dbtable", "earthquake_events_table") \
  .option("user", username) \
  .option("password", password) \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .mode("append") \
  .save()